In [1]:
import random
import json
from websocket_server import WebsocketServer

In [2]:
class Particle:
    def __init__(self, iden, x, y, theta):
        self.iden = iden
        self.x = x
        self.y = y
        self.theta = theta
        self.weight = 0.0
        self.associations = []
        self.sense_x = []
        self.sense_y = []


class ParticleFilter:
    n_particles = 10
    
    def __init__(self):
        self.particles = []
        self.is_initialized = False
        
    def init(self, sense_x, sense_y, sense_theta, sigma_pos):
        print("init", sense_x, sense_y, sense_theta)
        for i in range(self.n_particles):
            x = random.gauss(sense_x, sigma_pos[0])
            y = random.gauss(sense_y, sigma_pos[1])
            theta = random.gauss(sense_theta, sigma_pos[1])
            self.particles.append(Particle(i, x, y, theta))
            
        self.is_initialized = True
        
    def predict(self, delta_t, sigma_pos, previous_velocity, previous_yawrate):
        print("predict", delta_t, sigma_pos, previous_velocity, previous_yawrate)
        
    def find_best(self):
        best_weight = -1.0
        best_part = None
        
        for p in self.particles:
            if p.weight > best_weight:
                best_weight = p.weight
                best_part = p
        
        return p

In [ ]:
class SimulatorResponder:
    valid_prefix = '42'
    sigma_pos = [0.3, 0.3, 0.01]
    delta_t = 0.1
    
    def __init__(self, particle_filter):
        self.pf = particle_filter
    
    def on_connect(self, client, server):
        print("Client connected")
        
    def is_valid_message(self, message):
        return message.startswith(self.valid_prefix)
    
    def parse_data(self, message):
        data_start = len(self.valid_prefix)
        data = json.loads(message[data_start:])
        if data[0] == 'telemetry':
            return data[1]
        else:
            return None
        
    def best_particle(self):
        best = self.pf.find_best()
        return {
          'best_particle_associations': '',
          'best_particle_sense_x': '',
          'best_particle_sense_y': '',
          'best_particle_theta': best.theta,
          'best_particle_x': best.x,
          'best_particle_y': best.y
        }
        
    def respond(self, client, event, data):
        response = self.valid_prefix + json.dumps([event, data])
        client['handler'].send_message(response)
        
    def on_message(self, client, server, message):
        if not self.is_valid_message(message):
            return
        
        data = self.parse_data(message)
        if data is None: 
            self.respond(client, 'manual', {})
            return
        
        if not self.pf.is_initialized:
            sense_x = float(data['sense_x'])
            sense_y = float(data['sense_y'])
            sense_theta = float(data['sense_theta'])
            self.pf.init(sense_x, sense_y, sense_theta, self.sigma_pos)
        else:
            prev_velocity = float(data['previous_velocity'])
            prev_yawrate = float(data['previous_yawrate'])
            self.pf.predict(self.delta_t, self.sigma_pos, previous_velocity, previous_yawrate)

        self.respond(client, 'best_particle', self.best_particle())
        
    

In [ ]:
pf = ParticleFilter()
responder = SimulatorResponder(pf)    
server = WebsocketServer(4567, host='127.0.0.1')
server.set_fn_new_client(responder.on_connect)
server.set_fn_message_received(responder.on_message)
server.run_forever()

Listening on port 4567 for clients..
Client connected
init 6.2672 2.0931 0.0116
----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 63811)
----------------------------------------
Client connected


Traceback (most recent call last):
  File "/Users/serg_mo/miniconda3/envs/carnd-term1/lib/python3.5/socketserver.py", line 625, in process_request_thread
    self.finish_request(request, client_address)
  File "/Users/serg_mo/miniconda3/envs/carnd-term1/lib/python3.5/socketserver.py", line 354, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/Users/serg_mo/miniconda3/envs/carnd-term1/lib/python3.5/site-packages/websocket_server/websocket_server.py", line 136, in __init__
    StreamRequestHandler.__init__(self, socket, addr, server)
  File "/Users/serg_mo/miniconda3/envs/carnd-term1/lib/python3.5/socketserver.py", line 681, in __init__
    self.handle()
  File "/Users/serg_mo/miniconda3/envs/carnd-term1/lib/python3.5/site-packages/websocket_server/websocket_server.py", line 149, in handle
    self.read_next_message()
  File "/Users/serg_mo/miniconda3/envs/carnd-term1/lib/python3.5/site-packages/websocket_server/websocket_server.py", line 191, in rea

In [ ]:
sample_data = '42["telemetry",{"sense_x":"15.7544","sense_y":"5.0286","sense_theta":"0.2996","previous_velocity":"5.7666","previous_yawrate":"-0.0145","sense_observations_x":"-5.9299 -1.0073 22.1830 -3.1028 -14.4483 -29.1353 10.3003 -33.2641 -0.2889 44.5006 ","sense_observations_y":"4.3720 -9.4915 4.3068 -25.1741 -22.7379 3.9039 -38.8648 -31.5120 -46.1910 -16.4917 "}]'